In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%rm -rf sample_data

In [3]:
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
import h5py
import time
import sys
import tensorflow as tf
import cv2
import os
import pickle
import imutils

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [4]:
IMAGE_SIZE = 256
batch_size = 16

DATADIR_TRAIN = "/content/drive/My Drive/DetectBrainTumor/data_1_19052021/T2_Train/"
DATADIR_VAL = "/content/drive/My Drive/DetectBrainTumor/data_1_19052021/T2_Val/"
DATADIR_TRAIN_SAVE = "/content/drive/My Drive/DetectBrainTumor/data_15062021/T2_Train/"
DATADIR_VAL_SAVE = "/content/drive/My Drive/DetectBrainTumor/data_15062021/T2_Val/"

CATEGORIES = ["YES","NO"]

In [5]:
def crop_and_save_imgs (path, path_save, add_pixels_value, cate):
    """
    Finds the extreme points on the image and crops the rectangular out of them
    """
    for category in cate:
      path_new = path + category
      for img in os.listdir(path_new):
          # plt.imshow(img)
          # plt.show()
          img_array = cv2.imread(os.path.join(path_new, img))
          gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
          gray = cv2.GaussianBlur(gray, (5, 5), 0)

          # threshold the image, then perform a series of erosions +
          # dilations to remove any small regions of noise
          thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
          thresh = cv2.erode(thresh, None, iterations=2)
          thresh = cv2.dilate(thresh, None, iterations=2)

          # find contours in thresholded image, then grab the largest one
          cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
          cnts = imutils.grab_contours(cnts)
          c = max(cnts, key=cv2.contourArea)

          # find the extreme points
          extLeft = tuple(c[c[:, :, 0].argmin()][0])
          extRight = tuple(c[c[:, :, 0].argmax()][0])
          extTop = tuple(c[c[:, :, 1].argmin()][0])
          extBot = tuple(c[c[:, :, 1].argmax()][0])

          ADD_PIXELS = add_pixels_value
          new_img = img_array[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()

          filename = path_save + category + "/" + img 
          cv2.imwrite(filename, new_img)

In [6]:
print(DATADIR_TRAIN_SAVE)

/content/drive/My Drive/DetectBrainTumor/data_15062021/T2_Train/


In [7]:
crop_and_save_imgs(path=DATADIR_TRAIN, path_save=DATADIR_TRAIN_SAVE, add_pixels_value=5, cate=CATEGORIES)
crop_and_save_imgs(path=DATADIR_VAL, path_save=DATADIR_VAL_SAVE, add_pixels_value=5, cate=CATEGORIES)

In [8]:
# img = cv2.imread("IMG-0174-00010.jpg")
# gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
# gray = cv2.GaussianBlur(gray, (5, 5), 0)

# # threshold the image, then perform a series of erosions +
# # dilations to remove any small regions of noise
# thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
# thresh = cv2.erode(thresh, None, iterations=2)
# thresh = cv2.dilate(thresh, None, iterations=2)

# # find contours in thresholded image, then grab the largest one
# cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# cnts = imutils.grab_contours(cnts)
# c = max(cnts, key=cv2.contourArea)

# # find the extreme points
# extLeft = tuple(c[c[:, :, 0].argmin()][0])
# extRight = tuple(c[c[:, :, 0].argmax()][0])
# extTop = tuple(c[c[:, :, 1].argmin()][0])
# extBot = tuple(c[c[:, :, 1].argmax()][0])

# add_pixels_value = 5
# ADD_PIXELS = add_pixels_value
# img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()

# # cv2.drawContours(img, [c], -1, (0, 255, 255), 1)
# # cv2.circle(img, extLeft, 8, (0, 0, 255), -1)
# # cv2.circle(img, extRight, 8, (0, 255, 0), -1)
# # cv2.circle(img, extTop, 8, (255, 0, 0), -1)
# # cv2.circle(img, extBot, 8, (255, 255, 0), -1)
# # show the output image


# img = cv2.resize(img,(IMAGE_SIZE,IMAGE_SIZE))

# plt.imshow(img)
# plt.show()